In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.types import NumericType
from pyspark.sql.functions import col, count, when, unix_timestamp
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import Row

In [18]:
spark = SparkSession.builder.appName('CreditCardFraudEDA').getOrCreate()
fraudTrain = spark.read.csv('fraudTrain.csv', header=True, inferSchema=True)

In [19]:
fraudTest = spark.read.csv("fraudTest.csv", header=True, inferSchema=True)
drop_cols = ['_c0', 'cc_num', 'first', 'last', 'gender', 'street', 'city', 'state',
             'zip', 'job', 'dob', 'trans_num', 'merchant']

fraudTest = fraudTest.drop(*drop_cols)

In [20]:
fraudTest = fraudTest.withColumn("trans_date_ts", unix_timestamp("trans_date_trans_time")).drop("trans_date_trans_time")

In [21]:
fraudTrain.printSchema()
fraudTrain.show(5)
fraudTrain.describe().show()

root
 |-- _c0: integer (nullable = true)
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)

+---+---------------------+----------------+--------------------+-------------+------+---------+-------+--

Py4JJavaError: An error occurred while calling o821.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 437.0 failed 1 times, most recent failure: Lost task 9.0 in stage 437.0 (TID 4110) (10.12.70.111 executor driver): java.io.FileNotFoundException: C:\Users\hp\AppData\Local\Temp\blockmgr-09fbcc92-ee50-479a-8295-22be4934a6e7\25\temp_shuffle_d3c65b5b-05c5-4f5a-9ebb-3eea8475d2da (The system cannot find the path specified)
	at java.base/java.io.FileOutputStream.open0(Native Method)
	at java.base/java.io.FileOutputStream.open(FileOutputStream.java:295)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:236)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:147)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:167)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:330)
	at org.apache.spark.shuffle.sort.BypassMergeS

In [ ]:
schema = fraudTrain.schema.fields
column_types = {"numeric": [], "others": []}
for field in schema:
    if isinstance(field.dataType, NumericType):
        column_types["numeric"].append(field.name)
    else:
        column_types["others"].append(field.name)

print("Numeric : ", column_types["numeric"])
print("Others : ", column_types["others"])

Numeric :  ['_c0', 'cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud']
Others :  ['trans_date_trans_time', 'merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job', 'dob', 'trans_num']


In [ ]:
fraudTrain.select([count(when(col(c).isNull(), c)).alias(c) for c in fraudTrain.columns]).show()

+---+---------------------+------+--------+--------+---+-----+----+------+------+----+-----+---+---+----+--------+---+---+---------+---------+---------+----------+--------+
|_c0|trans_date_trans_time|cc_num|merchant|category|amt|first|last|gender|street|city|state|zip|lat|long|city_pop|job|dob|trans_num|unix_time|merch_lat|merch_long|is_fraud|
+---+---------------------+------+--------+--------+---+-----+----+------+------+----+-----+---+---+----+--------+---+---+---------+---------+---------+----------+--------+
|  0|                    0|     0|       0|       0|  0|    0|   0|     0|     0|   0|    0|  0|  0|   0|       0|  0|  0|        0|        0|        0|         0|       0|
+---+---------------------+------+--------+--------+---+-----+----+------+------+----+-----+---+---+----+--------+---+---+---------+---------+---------+----------+--------+



In [ ]:
drop_cols = ['_c0', 'cc_num', 'first', 'last', 'gender', 'street', 'city', 'state',
             'zip', 'job', 'dob', 'trans_num', 'merchant']
data = fraudTrain.drop(*drop_cols)

data = data.withColumn("trans_date_ts", unix_timestamp("trans_date_trans_time")).drop("trans_date_trans_time")

data.show(5)
data.describe().show()

+-------------+------+-------+---------+--------+----------+------------------+-----------+--------+-------------+
|     category|   amt|    lat|     long|city_pop| unix_time|         merch_lat| merch_long|is_fraud|trans_date_ts|
+-------------+------+-------+---------+--------+----------+------------------+-----------+--------+-------------+
|     misc_net|  4.97|36.0788| -81.1781|    3495|1325376018|         36.011293| -82.048315|       0|   1546281018|
|  grocery_pos|107.23|48.8878|-118.2105|     149|1325376044|49.159046999999994|-118.186462|       0|   1546281044|
|entertainment|220.11|42.1808| -112.262|    4154|1325376051|         43.150704|-112.154481|       0|   1546281051|
|gas_transport|  45.0|46.2306|-112.1138|    1939|1325376076|         47.034331|-112.561071|       0|   1546281076|
|     misc_pos| 41.96|38.4207| -79.4629|      99|1325376186|         38.674999| -78.632459|       0|   1546281186|
+-------------+------+-------+---------+--------+----------+------------------+-

In [ ]:
cat_col = "category"
indexer = StringIndexer(inputCol=cat_col, outputCol="category_index", handleInvalid='keep')
indexer_model = indexer.fit(data)
data_indexed = indexer_model.transform(data)

feature_cols = ['amt', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long', 'trans_date_ts', 'category_index']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
final_data = assembler.transform(data_indexed).select("features", "is_fraud")

final_data.show(5)

+--------------------+--------+
|            features|is_fraud|
+--------------------+--------+
|[4.97,36.0788,-81...|       0|
|[107.23,48.8878,-...|       0|
|[220.11,42.1808,-...|       0|
|[45.0,46.2306,-11...|       0|
|[41.96,38.4207,-7...|       0|
+--------------------+--------+
only showing top 5 rows



In [ ]:
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
lr = LogisticRegression(labelCol="is_fraud", featuresCol="features")
lr_model = lr.fit(train_data)
lr_preds = lr_model.transform(test_data)

In [ ]:
rf = RandomForestClassifier(labelCol="is_fraud", featuresCol="features", numTrees=50)
rf_model = rf.fit(train_data)
rf_preds = rf_model.transform(test_data)

In [ ]:
gbt = GBTClassifier(labelCol="is_fraud", featuresCol="features", maxIter=20)
gbt_model = gbt.fit(train_data)
gbt_preds = gbt_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="is_fraud")
print("LogReg AUC:", evaluator.evaluate(lr_preds))
print("Random Forest AUC:", evaluator.evaluate(rf_preds))
print("GBT AUC:", evaluator.evaluate(gbt_preds))

LogReg AUC: 0.8636127971215811
Random Forest AUC: 0.9140737744298402
GBT AUC: 0.9866982328734673


In [ ]:
user_input = {
    'amt': 100.0,
    'lat': 37.7749,
    'long': -122.4194,
    'city_pop': 50000,
    'unix_time': 1325376018,
    'merch_lat': 37.0,
    'merch_long': -122.0,
    'trans_date_ts': 1577836800,
    'category': 'misc_pos'
}

input_df = spark.createDataFrame([Row(**user_input)])
input_df = indexer_model.transform(input_df)
input_df = assembler.transform(input_df)
prediction = rf_model.transform(input_df).select("prediction", "probability")
prediction.show()

+----------+--------------------+
|prediction|         probability|
+----------+--------------------+
|       0.0|[0.99738403679790...|
+----------+--------------------+

